In [ ]:
import os
import sys
from dotenv import load_dotenv
from pathlib import Path

if sys.platform == 'linux':
    load_dotenv(dotenv_path=Path('.') / '.env.linux')
elif sys.platform == 'win32':
    load_dotenv(dotenv_path=Path('.') / '.env.win')
else:
    raise ValueError('Ваша операционная система не поддерживается')

os.environ['HF_HOME'] = os.getenv('HUGGING_FACE_CACHE_DIR')
DATASET_PATH = os.getenv('DATASET_PATH', None)
print(DATASET_PATH)
DATASET_FOLDER = os.getenv('DATASET_FOLDER_PATH', None)
MODEL_NAME = 'Qwen/Qwen3-4B-Instruct-2507-recepies'
MODEL_SIZE = os.getenv('MODEL_SIZE', None)
MODEL_SAVE_DIR=os.getenv('MODEL_SAVE_DIR', None)
MODEL_SAVE_NAME = os.getenv('MODEL_SAVE_NAME', None)
print(MODEL_NAME)

/mnt/12A4CA9DA4CA8329/Files/Datasets/recipes_generation/fine_tuning_preprocessed.csv
Qwen/Qwen3-4B-Instruct-2507


In [2]:
from peft import LoraConfig, TaskType, get_peft_model
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from datasets import load_from_disk, load_dataset

In [3]:
peft_config = LoraConfig(
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj", "o_proj", "k_proj", "up_proj", "down_proj", "gate_proj"],
    bias="none",
    task_type="CAUSAL_LM")

In [4]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=False,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    dtype=torch.bfloat16,
    trust_remote_code=True)

model = get_peft_model(base_model, peft_config)

# Количество обучаемых параметров
model.print_trainable_parameters()
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

trainable params: 66,060,288 || all params: 4,088,528,384 || trainable%: 1.6157


In [5]:
full_dataset = load_dataset('csv', data_files=[DATASET_PATH])
full_dataset = full_dataset.remove_columns(['composition_list', 'prompt'])

In [6]:
def get_assistant_content(row):
    return {'assistant_content' :\
f'''Ингридиенты:
{row['ingredients']}

Процесс приготовления:
{row['instructions']}'''}

full_dataset = full_dataset.map(get_assistant_content, batched=False)

In [7]:
# Preprocesses the dataset
def preprocess_dataset_with_eos(eos_token, dataset):
    def preprocess_function(examples):
        instructions = examples["assistant_content"]
        names = examples["name"]

        chats = []
        for recipe, name in zip(instructions, names):
            # Append the EOS token to the response
            recipe += eos_token

            chat = [
                {"role": "user", "content": f"Как я могу приготовить {name}?"},
                {"role": "assistant", "content": recipe},
            ]

            chats.append(chat)
        return {"messages": chats}
    dataset = dataset.map(preprocess_function, batched=True, remove_columns=dataset['train'].column_names)
    return dataset

In [8]:
EOS_TOKEN = tokenizer.eos_token
dataset = full_dataset['train'].train_test_split(test_size=0.1)
dataset = preprocess_dataset_with_eos(EOS_TOKEN, dataset)

Map:   0%|          | 0/25095 [00:00<?, ? examples/s]

Map:   0%|          | 0/2789 [00:00<?, ? examples/s]

In [9]:
def formatting_function(examples):
    return { 'chat' : tokenizer.apply_chat_template(
        examples["messages"],
        tokenize=False,
        add_generation_prompt=False)}

dataset = dataset.map(formatting_function, batched=True).remove_columns('messages')

Map:   0%|          | 0/25095 [00:00<?, ? examples/s]

Map:   0%|          | 0/2789 [00:00<?, ? examples/s]

In [10]:
dataset['train'][0]

{'chat': '<|im_start|>user\nКак я могу приготовить Шербет из\xa0ревеня?<|im_end|>\n<|im_start|>assistant\nИнгридиенты:\n Сахар 0.2 стак.\n Вода 0.2 стак.\n Лимонный сок 0.5 столов.\n Стебли ревеня 75.0 гр.\n Кукурузный сироп 0.3 столов.\n\nПроцесс приготовления:\n1. В небольшой кастрюле смешайте сахар, воду и лимонный сок. Поставьте на маленький огонь и перемешивайте, пока сахар не растворится. Увеличьте огонь и доведите до кипения.\n2. Добавьте нарезанный на маленькие кусочки ревень и варите до мягкости примерно 10 минут.\n3. Переложите массу в комбайн и измельчите до состояния пюре.\n4. Добавьте кукурузный сироп и поставьте охлаждаться на 1 час.\n5. Переложите массу в мороженицу и заморозьте.<|im_end|><|im_end|>\n'}

In [11]:
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False) # Отключаем masked language modelling (mlm)

In [12]:
tokenized_dataset = dataset.map(lambda x: tokenizer(x['chat'], padding=True, truncation=True, max_length=680), batched=True).remove_columns('chat')

Map:   0%|          | 0/25095 [00:00<?, ? examples/s]

Map:   0%|          | 0/2789 [00:00<?, ? examples/s]

In [13]:
args = TrainingArguments(
    output_dir=os.path.join(MODEL_SAVE_DIR, 'training_checkpoints'),  # Кастомная директория
    overwrite_output_dir=True,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.01,
    warmup_steps=200,
    lr_scheduler_type="linear",
    learning_rate=1e-4,
    save_steps=200,
    eval_steps=200,
    logging_steps=200,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,  # Загружать лучшую модель
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    bf16=torch.cuda.is_available(),  # Только если GPU поддерживает
    remove_unused_columns=False,
    gradient_checkpointing=False,  # Экономия памяти
    save_total_limit=2,  # Хранить только 2 лучших чекпоинта
    push_to_hub=False,
    report_to="none",  # Отключить логирование в сторонние сервисы
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

/tmp/ipykernel_30302/4267563310.py:28: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
# Обучение
print("Начало обучения...")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151643}.


Начало обучения...


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

Переделать сохранение модели!!!

In [ ]:
# Сохранение финальной модели
trainer.save_model(os.path.join(MODEL_SAVE_DIR, MODEL_SAVE_NAME))
tokenizer.save_pretrained(os.path.join(MODEL_SAVE_DIR, MODEL_SAVE_NAME))

print(f'Модель сохранена в {os.path.join(MODEL_SAVE_DIR, MODEL_SAVE_NAME)}')